In [1]:
from datetime import date
from pandas import json_normalize
import pandas as pd
pd.set_option('display.max_columns', None)

In [2]:
df_work_raw = catalog.load('raw/openalex/work#parquet')

                    INFO     Loading data from raw/openalex/work#parquet (ParquetDataset)...    ]8;id=510439;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=595601;file:///home/pablo/dev/CIC/dw-cic/kedro/kedro-cic/.venv/lib/python3.10/site-packages/kedro/io/data_catalog.py#389\389]8;;\

In [3]:
df_work_raw = df_work_raw.convert_dtypes()
df_work_raw = df_work_raw.loc[:,['id','locations']]

In [4]:
df_work_raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1200 entries, 0 to 199
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         1200 non-null   string
 1   locations  1200 non-null   object
dtypes: object(1), string(1)
memory usage: 28.1+ KB


In [5]:
df_work_raw

,id,locations
0,https://openalex.org/W2089852843,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
1,https://openalex.org/W3037375387,"[{'is_accepted': True, 'is_oa': True, 'is_publ..."
2,https://openalex.org/W2032143374,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
3,https://openalex.org/W2021071494,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
4,https://openalex.org/W1992226558,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
...,...,...
195,https://openalex.org/W2080847848,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
196,https://openalex.org/W2129200480,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
197,https://openalex.org/W1969965905,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."
198,https://openalex.org/W1997756942,"[{'is_accepted': False, 'is_oa': False, 'is_pu..."


In [6]:
df_work_authorship = df_work_raw.explode('locations').reset_index(drop=True)

In [7]:
def openalex_land_work_location(df_work_raw):

    df_work_location = df_work_raw.explode('locations').reset_index(drop=True)

    df_work_location = pd.concat([df_work_location['id'], json_normalize(df_work_location['locations'])], axis=1)

    df_work_location.columns = df_work_location.columns.str.replace('.', '_')

    df_work_location = df_work_location[[
        'id', 
        'source_id', 'source_display_name', 'source_is_core', 'source_type',
        'source_host_organization', 'source_host_organization_name',
        'is_accepted', 'is_oa', 'is_published', 'landing_page_url',
        'license', 'license_id', 'pdf_url', 'version',
        # 'source_host_organization_lineage', 'source_host_organization_lineage_names', 'source_issn',
        'source_is_in_doaj', 'source_is_oa', 'source_issn_l'
    ]]

    df_work_location['load_datetime'] = date.today()

    return df_work_location


In [8]:
stage_work_location = openalex_land_work_location(df_work_raw)
stage_work_location

,id,source_id,source_display_name,source_is_core,source_type,source_host_organization,source_host_organization_name,is_accepted,is_oa,is_published,landing_page_url,license,license_id,pdf_url,version,source_is_in_doaj,source_is_oa,source_issn_l,load_datetime
0,https://openalex.org/W2089852843,https://openalex.org/S24688465,Physica A Statistical Mechanics and its Applic...,True,journal,https://openalex.org/P4310320990,Elsevier BV,False,False,False,https://doi.org/10.1016/s0378-4371(98)00437-3,None,None,None,None,False,False,0378-4371,2025-02-01
1,https://openalex.org/W3037375387,https://openalex.org/S143381477,Environment International,True,journal,https://openalex.org/P4310320990,Elsevier BV,True,True,True,https://doi.org/10.1016/j.envint.2020.105876,cc-by-nc-nd,https://openalex.org/licenses/cc-by-nc-nd,None,publishedVersion,True,True,0160-4120,2025-02-01
2,https://openalex.org/W3037375387,https://openalex.org/S4306401280,DOAJ (DOAJ: Directory of Open Access Journals),False,repository,None,None,False,False,False,https://doaj.org/article/9410c9207e9e450e8f655...,None,None,None,None,False,True,None,2025-02-01
3,https://openalex.org/W3037375387,https://openalex.org/S4306402455,CONICET Digital (CONICET),False,repository,https://openalex.org/I4210148385,Biomedicine Research Institute of Buenos Aires...,True,True,True,https://hdl.handle.net/11336/142501,cc-by-nc-nd,https://openalex.org/licenses/cc-by-nc-nd,https://ri.conicet.gov.ar/bitstream/11336/1425...,publishedVersion,False,True,None,2025-02-01
4,https://openalex.org/W3037375387,https://openalex.org/S4306525036,PubMed,False,repository,https://openalex.org/I1299303238,National Institutes of Health,False,False,False,https://pubmed.ncbi.nlm.nih.gov/32590284,None,None,None,None,False,False,None,2025-02-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2826,https://openalex.org/W1969965905,https://openalex.org/S4306402455,CONICET Digital (CONICET),False,repository,https://openalex.org/I4210148385,Biomedicine Research Institute of Buenos Aires...,True,True,True,https://hdl.handle.net/11336/79606,cc-by-nc-sa,https://openalex.org/licenses/cc-by-nc-sa,https://ri.conicet.gov.ar/bitstream/11336/7960...,publishedVersion,False,True,None,2025-02-01
2827,https://openalex.org/W1969965905,https://openalex.org/S4306525036,PubMed,False,repository,https://openalex.org/I1299303238,National Institutes of Health,False,False,False,https://pubmed.ncbi.nlm.nih.gov/20416796,None,None,None,None,False,False,None,2025-02-01
2828,https://openalex.org/W1997756942,https://openalex.org/S184777250,IEEE Transactions on Industrial Informatics,True,journal,https://openalex.org/P4310319808,Institute of Electrical and Electronics Engineers,False,False,False,https://doi.org/10.1109/tii.2012.2221470,None,None,None,None,False,False,1551-3203,2025-02-01
2829,https://openalex.org/W1997756942,https://openalex.org/S4306402455,CONICET Digital (CONICET),False,repository,https://openalex.org/I4210148385,Biomedicine Research Institute of Buenos Aires...,True,True,True,https://hdl.handle.net/11336/1337,cc-by-nc-nd,https://openalex.org/licenses/cc-by-nc-nd,https://ri.conicet.gov.ar/bitstream/11336/1337...,publishedVersion,False,True,None,2025-02-01
